# 데이터 수집

In [2]:
import requests
import pandas as pd
import time

## 3초 꿀팁) 복붙으로 1초만에 데이터프레임 만들기
[코스피 일별 시세](https://finance.naver.com/sise/sise_index.naver?code=KOSPI)를 복사해 주세요.


In [43]:
pd.read_clipboard()

,날짜,체결가,전일비,등락률,거래량(천주),거래대금(백만)
0,2023.08.11,"2,591.26",하락 10.30,-0.40%,"675,390","10,513,466"
1,2023.08.10,"2,601.56",하락 3.56,-0.14%,"568,155","11,349,005"
2,2023.08.09,"2,605.12",상승 31.14,+1.21%,"467,527","10,326,262"
3,2023.08.08,"2,573.98",하락 6.73,-0.26%,"638,419","12,435,382"
4,2023.08.07,"2,580.71",하락 22.09,-0.85%,"504,071","11,894,471"
5,2023.08.04,"2,602.80",하락 2.59,-0.10%,"645,750","10,971,353"


## 1) REST API

API를 사용하기 위해서는 웹에서 정보를 요청하는 `requests` 패키지가 필요합니다.

`request.get()`패키지로 URL에 대한 응답을 요청하면 웹에서 그 주소에 해당하는 응답을 제공합니다.

REST API로 정보를 제공하는 웹페이지들은 URL을 어떻게 짜야 하는지, 사용 메뉴얼을 제공하고 있습니다.

금감원 산하 금융통계정보시스템에서 제공하는 "통계정보 API"를 사용해 보겠습니다.

URL 생성 규칙은 [여기](https://fisis.fss.or.kr/fss/fsiview/indexw.html)서 알 수 있습니다.
```
auth: 인증키
financeCd: 은행 코드
listNo: 통계번호 코드
startBaseMm: 시작 시점 연월
endBaseMm: 종료 시점 연월
```

* 인증키란, 누가 정보를 요청했는지 알기 위해 사용하는 일종의 보안코드입니다. 인증키를 기준으로 이 정보를 보안상 제공해줄수 있는지/없는지, 하루 이용 한도 몇 회 중 몇 회까지 사용하였는지, 이 API를 부적절한 목적으로 사용하지는 않는지 확인하는 데 사용합니다. **본인의 인증키는 절대로 외부에 유출해서는 안 됩니다.**

API를 통해서 URL을 요청하면 웹페이지는 `XML` 혹은 `JSON`과 같은 텍스트 파일을 보내 줍니다. 각 자료형에 대한 설명은 굳이 하지 않겠습니다.

개인적으로 json포맷이 훨씬 정리가 편하다고 생각되어서, 오늘은 json포맷으로 불러오는 방법을 알아보겠습니다.

오늘 실습의 내용은 "우리은행의 자동화기기 설치현황 가져오기 입니다.

이 웹페이지는 URL 예시를 제공하고 있기 때문에, 예시를 참고하여 URL을 구성하면 됩니다.

우리은행의 은행 코드는 0010001, 자동화기기 설치현황 통계 코드는 SA026입니다.

이것도 목록을 반환해 주는 API 혹은 API테스트 화면 등으로 해당 코드번호를 받아 오는 방법으로 찾은 것이며, 찾는 과정은 시간관계상 스킵하겠습니다.

In [38]:
#제 인증키는 지워 뒀슴다
url="https://fisis.fss.or.kr/openapi/statisticsInfoSearch.json?lang=kr&auth=00000000000000000000000000000000&financeCd=0010001&listNo=SA026&term=H&startBaseMm=202001&endBaseMm=202306"

`requests.get()` 으로 url의 응답을 요청할 수 있습니다.

In [39]:
response=requests.get(url)
response

<Response [200]>

In [40]:
response.json()

{'result': {'err_cd': '000',
  'err_msg': '정상',
  'total_count': '42',
  'description': [{'column_id': 'a', 'column_nm': '수'}],
  'date_of_settlement': '12/31',
  'unit': '개',
  'list': [{'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리은행',
    'account_cd': 'A',
    'account_nm': 'CD',
    'a': '0'},
   {'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리은행',
    'account_cd': 'B',
    'account_nm': 'ATM',
    'a': '4727'},
   {'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리은행',
    'account_cd': 'C',
    'account_nm': '기타',
    'a': '905'},
   {'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리은행',
    'account_cd': 'E',
    'account_nm': '화상단말기',
    'a': '46'},
   {'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리은행',
    'account_cd': 'D',
    'account_nm': '합계',
    'a': '5678'},
   {'base_month': '202006',
    'finance_cd': '0010001',
    'finance_nm': '우리

In [41]:
response.json()['result']['list']

[{'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'A',
  'account_nm': 'CD',
  'a': '0'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'B',
  'account_nm': 'ATM',
  'a': '4727'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'C',
  'account_nm': '기타',
  'a': '905'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'E',
  'account_nm': '화상단말기',
  'a': '46'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'D',
  'account_nm': '합계',
  'a': '5678'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'F',
  'account_nm': '무인점포',
  'a': '182'},
 {'base_month': '202006',
  'finance_cd': '0010001',
  'finance_nm': '우리은행',
  'account_cd': 'G',
  'account_nm': '점내 365일 코너',
  'a': '880'},
 {'base_month': '202012',
  'finance_cd': 

In [42]:
pd.json_normalize(response.json()['result']['list'])

,base_month,finance_cd,finance_nm,account_cd,account_nm,a
0,202006,0010001,우리은행,A,CD,0
1,202006,0010001,우리은행,B,ATM,4727
2,202006,0010001,우리은행,C,기타,905
3,202006,0010001,우리은행,E,화상단말기,46
4,202006,0010001,우리은행,D,합계,5678
5,202006,0010001,우리은행,F,무인점포,182
6,202006,0010001,우리은행,G,점내 365일 코너,880
7,202012,0010001,우리은행,A,CD,0
8,202012,0010001,우리은행,B,ATM,4487
9,202012,0010001,우리은행,C,기타,861


## 2) Selenium

셀레늄의 원래 목적은 "웹 자동화"입니다. 웹 페이지에서 사용자가 조작해야 하는 기능들 (마우스 클릭, 드래그, 키보드 입력) 등을 모두 코드로 자동화해 사용자의 수고를 덜어 주는 것이 원래의 목적입니다.

하지만 이를 이용하면 강력한 웹 크롤러 개발이 가능합니다. 로그인이 필요하다거나, 복잡한 조작이 필요한 상황에서 매우 강력한 힘을 발휘합니다.

셀레늄을 사용하기 위해서는 웹 페이지를 컨트롤하는 보조 프로그램인 **웹 드라이버**가 필요합니다. 

대부분 사용하고 계실 chorme과 Edge 모두 웹 드라이버를 제공하고 있으며, safari에서도 사용이 가능합니다.

오늘은 가장 일반적으로 사용되는 크롬을 사용하겠습니다.

In [ ]:
#셀레늄이 설치되어 있지 않다면..
# !pip install selenium

In [3]:
from selenium import webdriver #웹 페이지를 조작하는 webdriver 프로그램을 다루기 위한 모듈

오늘 셀레늄 실습은 "키움증권에서 최근 6개월간 발행된 ELS상품 목록 수집하기"를 수행해 보겠습니다.

In [56]:
driver=webdriver.Chrome("/Users/yihoon-j/Documents/fr/2학기 교안 작업/chromedriver") #웹 드라이버를 읽어 옵니다.
url="https://www1.kiwoom.com/h/main" #컨트롤하고자 하는 URL을 가져 옵니다. 그냥 ELS조회 화면으로 바로 들어가도 되지만, 실습을 위해 해당 화면에 접근하는 과정부터 자동화해 보겠습니다.
driver.get(url)

/var/folders/_x/6zxfdhhj0cn8q6l5rd4411y80000gn/T/ipykernel_10244/1113974478.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("/Users/yihoon-j/Documents/fr/2학기 교안 작업/chromedriver") #웹 드라이버를 읽어 옵니다.


**크롬 창이 생겼다면 절대 닫지 말고 주피터로 돌아옵시다**

*혹시 중간에 창을 닫을 경우 드라이버가 담고 있는 웹페이지 정보가 사라지기 때문에, 위의 셀부터 다시 실행해서 접근하셔야 합니다.*

그리고, 셀레늄을 사용할 때는 웹 페이지를 최대화시키는 것이 좋습니다.
창의 크기가 달라지면 버튼의 위치가 달라져 조작이 불가능한 경우가 생기기 때문에, 보통 최대 사이즈로 창을 맞추어 두고 작업하는 경우가 일반적입니다.

In [35]:
driver.maximize_window()

**click**

일단 팝업에 막혀서 아무것도 클릭이 되지 않네요. 팝업을 닫아 봅시다. 팝업의 닫기 버튼을 클릭하면 되겠죠?

아까 알아본 것처럼, 웹 페이지는 결국 문서를 시각화한 것이기 때문에, 문서에서 그 버튼이 어디 있는지 찾으면 됩니다.

셀레늄을 통해 버튼의 위치에 접근하는 다양한 방법이 있지만 가장 편하고 직관적인 방법은 xpath라는 고유 주소를 찾는 방법입니다.

먼저 웹브라우저에서 F12를 열어 개발자 도구에 진입합니다.
그 후 팝업의 `닫기` 버튼에서 `우클릭` - `검사`를 누릅니다.

이때 개발자 도구의 특정 섹션이 하이라이트되는 것을 볼 수 있습니다. 해당 부분에서 우클릭 - `Copy` - `Copy XPath`를 눌러 줍시다.

xpath를 입력할 때는 반드시 작은따옴표를 사용하도록 합니다.

In [57]:
driver.find_element('xpath', '//*[@id="NotiClose"]').click()
#팝업이 뜨지 않는 경우도 있기 때문에 try-except문으로 짜는 게 좋다.

In [59]:
# "전체메뉴" 클릭
driver.find_element('xpath', '//*[@id="btn_all_menu"]').click()

# "금융상품" 클릭
driver.find_element('xpath', '//*[@id="kwsAllMenu"]/div/div[1]/ul/li[3]/a').click()

# "공지사항/가격정보" 클릭
driver.find_element('xpath', '//*[@id="kwsAllMenu"]/div/div[1]/div/section[3]/ul[2]/li[1]/ul/li[4]/a').click()
time.sleep(1) #여기서 1초 쉬고 진행

# "청약종료상품" 클릭
driver.find_element('xpath', '//*[@id="kwsAllMenu"]/div/div[1]/div/section[3]/ul[2]/li[1]/ul/li[4]/ul/li[2]/a').click()

# "ELS" 클릭
driver.find_element('xpath', '//*[@id="eventList"]/li[2]/span/label').click()

# "원금비보장형" 클릭
driver.find_element('xpath', '//*[@id="princTypeList"]/li[4]/span/label').click()

# "6개월" 클릭
driver.find_element('xpath', '//*[@id="searchForm"]/div[2]/ul/li[4]/div[1]/div/div/div/span[3]/label').click()

**send_keys**

키워드를 입력해야 하는 경우가 발생할 수 있습니다.

예를 들어, 키움에서 판매하는 종목형 ELS는 '뉴글로벌'이라는 명칭이 붙습니다.

해당 ELS만을 검색해 봅시다.

In [60]:
driver.find_element('xpath', '//*[@id="searchForm"]/div[2]/ul/li[4]/div[2]/div/div/span/input').send_keys('뉴글로벌')

#'검색' 클릭
driver.find_element('xpath', '//*[@id="endSearchBtn"]').click()

조회되는 ELS가 많은 경우 '더보기'를 눌러 더 많은 ELS를 조회해야 합니다. '더보기'를 반복적으로 눌러 전체 목록이 뜨게 해야 합니다.

이때 해당 버튼을 몇 번 눌러야 할 지 알 수 없으므로, 반복문과 try-except 문을 조하하여 더이상 누를 수 없을 때까지 반복해서 누르면 됩니다.

이때 '더보기'버튼을 누르고 새 목록이 뜰 때까지 약간의 딜레이가 발생하므로, '더보기' 버튼을 누르면 1초씩 쉬어 주면서 눌러 줍시다.


In [8]:
while True:
    try:
        driver.find_element('xpath', '//*[@id="contentMain"]/div/div[3]/div[2]/button/span').click()
        print('더보기 버튼을 눌렀습니다.')
        time.sleep(2)
    except:
        break
    
print('더이상 버튼을 누를 수 없습니다. 전체 데이터가 조회되었습니다.')
    

더보기 버튼을 눌렀습니다.
더보기 버튼을 눌렀습니다.
더보기 버튼을 눌렀습니다.
더보기 버튼을 눌렀습니다.
더이상 버튼을 누를 수 없습니다. 전체 데이터가 조회되었습니다.


**text**
원하는 화면까지 왔다면 이제 정보를 수집해야겠죠? text함수로 텍스트를 가져올 수 있습니다.

첫 번째 ELS의 정보를 수집해 봅시다.

In [10]:
#'검사'로 찾은 xpath가 먹히지 않을 때는 그 상위 아이템의 xpath를 가져오면 해결되는 경우가 대부분.
name=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[1]/div/div[2]').text #종목명
base=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[2]').text #기초자산
maxgain=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[3]').text #조건충족시 수익률
maxloss=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[4]').text #조건미충족시 수익률
els_type=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[5]').text #유형
issu_date=driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[6]').text #발행일
expire_date=driver.find_element('xpath','//*[@id="endList"]/tr[2]/td[1]').text #만기일
knockin= driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[7]').text #KI여부
returned= driver.find_element('xpath','//*[@id="endList"]/tr[2]/td[2]').text #상환여부
early_returned= driver.find_element('xpath','//*[@id="endList"]/tr[1]/td[8]').text.replace('\n상세보기','') #조기상환 결정여부: 뒤에 상세보기랑 같이 나와서...
print(name, base, maxgain, maxloss, els_type, issu_date, expire_date, knockin, returned, early_returned, sep='\n')

제812회뉴글로벌100조파생결합증권(주가연
S&P500, 테슬라
최대 연 14.01%
-100%
만기2년 월지급식 조기상환형(90-85-80-65)40KI,월지급배리어50
2023.08.11
2025.08.13
아니요
미상환
미결정


In [11]:
#이렇게 데이터프레임으로 변환도 가능하겠죠?
df=pd.DataFrame([name, base, maxgain, maxloss, els_type, issu_date, expire_date, knockin, returned, early_returned]).T
df.columns=['종목명', '기초자산', '조건충족시 수익률', '조건미충족시 수익률', '유형', '발행일','만기일', 'KI여부', '상환여부', '조기상환 결정여부']
df

,종목명,기초자산,조건충족시 수익률,조건미충족시 수익률,유형,발행일,만기일,KI여부,상환여부,조기상환 결정여부
0,제812회뉴글로벌100조파생결합증권(주가연,"S&P500, 테슬라",최대 연 14.01%,-100%,"만기2년 월지급식 조기상환형(90-85-80-65)40KI,월지급배리어50",2023.08.11,2025.08.13,아니요,미상환,미결정


이제 이 코드를 반복문으로 바꾸면 모든 정보를 수집할 수 있겠죠?

In [30]:
full_els=pd.DataFrame(columns=['종목명', '기초자산', '조건충족시 수익률', '조건미충족시 수익률', '유형', '발행일','만기일', 'KI여부', '상환여부', '조기상환 결정여부'])
i=1
while True:
    try:
        name=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[1]/div/div[2]').text #종목명
        base=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[2]').text #기초자산
        maxgain=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[3]').text #조건충족시 수익률
        maxloss=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[4]').text #조건미충족시 수익률
        els_type=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[5]').text #유형
        issu_date=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[6]').text #발행일
        expire_date=driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i)+']/td[1]').text #만기일
        knockin= driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[7]').text #KI여부
        returned= driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i)+']/td[2]').text #상환여부
        early_returned= driver.find_element('xpath','//*[@id="endList"]/tr['+str(2*i-1)+']/td[8]').text.replace('\n상세보기','') #조기상환 결정여부
        temp=pd.DataFrame([name, base, maxgain, maxloss, els_type, issu_date, expire_date, knockin, returned, early_returned]).T
        temp.columns=['종목명', '기초자산', '조건충족시 수익률', '조건미충족시 수익률', '유형', '발행일','만기일', 'KI여부', '상환여부', '조기상환 결정여부']
        full_els=pd.concat([full_els, temp])
        i+=1
    except:
        break
full_els=full_els.reset_index(drop=True)
print(f'총 {i-1}개의 ELS 정보를 수집하였습니다')

총 174개의 ELS 정보를 수집하였습니다


In [31]:
full_els

,종목명,기초자산,조건충족시 수익률,조건미충족시 수익률,유형,발행일,만기일,KI여부,상환여부,조기상환 결정여부
0,제812회뉴글로벌100조파생결합증권(주가연,"S&P500, 테슬라",최대 연 14.01%,-100%,"만기2년 월지급식 조기상환형(90-85-80-65)40KI,월지급배리어50",2023.08.11,2025.08.13,아니요,미상환,미결정
1,제811회뉴글로벌100조파생결합증권(주가연,"테슬라, AMD",최대 연 23.00%,-100%,만기1년 조기상환형 (85-80-70-65) 45KI,2023.08.11,2024.08.14,아니요,미상환,미결정
2,제810회뉴글로벌100조파생결합증권(주가연,"테슬라, 엔비디아",최대 연 21.30%,-100%,만기2년 조기상환형 (85-80-75-75-70-60) 40KI,2023.08.11,2025.08.13,아니요,미상환,미결정
3,제809회뉴글로벌100조파생결합증권(주가연,"테슬라, 엔비디아",최대 연 18.00%,-100%,만기2년 조기상환형 (75-75-75-70-70-60) 40KI,2023.08.11,2025.08.13,아니요,미상환,미결정
4,제808회뉴글로벌100조파생결합증권(주가연,"AMD, 엔비디아",최대 연 15.00%,-100%,만기2년 조기상환형 (80-80-75-75-70-60) 40KI,2023.08.11,2025.08.13,아니요,미상환,미결정
...,...,...,...,...,...,...,...,...,...,...
169,제621회뉴글로벌100조파생결합증권(주가연,"테슬라, 엔비디아",최대 연 25.4%,-100%,만기1년 조기상환형 (80-80-70-65) 40KI,2023.02.17,2024.02.21,아니요,상환,결정
170,제620회뉴글로벌100조파생결합증권(주가연,"테슬라, AMD",최대 연 26.3%,-100%,만기2년 조기상환형 (85-80-80-75-75-70-70-65) 40KI,2023.02.17,2025.02.21,아니요,미상환,미결정
171,제619회뉴글로벌100조파생결합증권(주가연,"테슬라, 엔비디아",최대 연 24.4%,-100%,만기2년 리자드조기상환형 (85-80-80-75(L50)-75-70-70-65)40KI,2023.02.17,2025.02.21,아니요,미상환,미결정
172,제617회뉴글로벌100조파생결합증권(주가연,"S&P500, 테슬라",최대 연 21%,-100%,만기2년 조기상환형 (85-80-80-75-75-70-70-60) 35KI,2023.02.17,2025.02.19,아니요,미상환,미결정


**다른 창을 열어서 조작해야 하는 경우**

셀레늄의 단점은, 어떤 동작을 했을 때, 새로운 창이 열리고, 그 창을 컨트롤해야 한다면 다소 번거로운 조작이 필요하다는 점입니다.

예를 들어, 현재 화면에서 ELS의 청약경쟁률을 보고자 한다면, 해당 ELS 상세 정보가 담긴 팝업창을 띄워서 거기서 조작을 해야 합니다.

또 조작을 마친 후에는 다시 원래의 ELS 목록 창으로 돌아와야 하겠죠.

시간 관계상 간단한 조작 하나만 해 보겠습니다.

In [61]:
driver.current_url

'https://www1.kiwoom.com/wm/edl/es020/edlEndElsView?dummyVal=0'

이 코드는 셀레늄 공식 도큐먼트에 있기 때문에, 굳이 알아두실 필요 없습니다. 저도 못 외워요.

In [62]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)
original_window = driver.current_window_handle #현재 드라이버가 건드리고 있는 윈도우를 기억해 둡니다

assert len(driver.window_handles) == 1
driver.find_element('xpath','//*[@id="data-stkcode"]').click() #새 창을 띄우는 동작

wait.until(EC.number_of_windows_to_be(2))
for window_handle in driver.window_handles:
    if window_handle != original_window:
        driver.switch_to.window(window_handle)
        break


In [63]:
#창이 바뀌었음을 볼 수 있습니다.
driver.current_url

'https://www1.kiwoom.com/wm/edl/es010/fndElsDetailPopup'

In [64]:
driver.find_element('xpath','//*[@id="subsButton"]').click() #청약율이 뜨는 팝업
compet=driver.find_element('xpath','//*[@id="listTbody"]/tr/td[4]').text #청약율
print('경쟁률:',compet)

0.0894 : 1


In [ ]:
driver.close()
driver.switch_to.window(original_window)

In [66]:
driver.current_url #원래의 url로 돌아옴

'https://www1.kiwoom.com/wm/edl/es020/edlEndElsView?dummyVal=0'

참고) 셀레늄을 활용한 크롤러를 사용하는 경우는 주로
* 데이터를 가져올 수 있는 다른 뚜렷한 방법이 없는 경우: 복붙이나 엑셀 다운이 된다면 그냥 그걸로 하세요...
* 로그인 없이는 접근이 어려운 경우: 로그인한 후에만 정보를 읽을 수 있는 경우에는 셀레늄이 필요합니다.
* 크롤링 차단 기능이 존재하는 웹페이지를 파싱하는 경우: html 파싱을 막아 둔 페이지도 정보를 가져올 수 있습니다.
* 작업 자동화가 필요한 경우: 정기적/비정기적으로 똑같은 작업을 반복적으로 수행해야 할 경우, 수집 프로세스를 자동화하는 툴로 셀레늄을 우선 고려할 필요가 있습니다.

In [66]:
pd.read_clipboard()

ParserError: Expected 5 fields in line 8, saw 12. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.